In [48]:
import pandas as pd
import numpy as np
import glob
import gzip
import shutil
import os
from tqdm import tqdm

# set infinite display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


In [49]:
print(os.getcwd())

/Users/worker/Desktop/capstone/bin/Capstone


In [50]:
# convert tsv into dataframe
df = pd.read_csv('../../data/CPTAC-3.bio_repl_query.sur.tsv', sep='\t', header=0)
df2 = pd.read_csv('../../data/CPTAC-3.gdc_repl_query.tsv', sep='\t', header=0)

In [51]:
# get shape of the dataframe
print(df.shape)
# get shape of the dataframe
print(df2.shape)

(6702, 22)
(1219, 117)


In [52]:
# check duplicated case_id in df
print(df.case_id.duplicated().sum())
# check duplicated case_id in df2
print(df2.case_id.duplicated().sum())

4633
0


In [53]:
# preview dataframe
df.head()

,case_id,aliquot_id,read_group_id,submitted_unaligned_reads_id,disease_type,has_blood_cancer,primary_site,project_id,tissue_type,instrument_model,RIN,includes_spike_ins,library_preparation_kit_name,library_preparation_kit_vendor,library_preparation_kit_version,library_preparation_kit_catalog_number,library_selection,library_strand,library_strategy,size_selection_range,spike_ins_concentration,read_pair_number
0,5705efcc-b48f-435c-8a28-9e0d407ecadd,75ac0619-947a-427b-a53f-71e121a7ec8f,71894d8b-5210-44dc-aadc-a199d3843dd2,8fcdd6a1-a7c7-41b5-8b44-e41f2237b236,Adenomas and Adenocarcinomas,False,"Uterus, NOS",CPTAC-3,Tumor,Illumina HiSeq 4000,NaN,True,TruSeq Stranded Total RNA Library Prep Kit wit...,Illumina,NaN,NaN,rRNA Depletion,First_Stranded,RNA-Seq,308.0,1.0,R1
1,763e0702-8379-4b5e-95d1-a84f412c51e7,ce810e2e-4929-4bbc-95ff-6da493477391,c2980255-7c57-4b79-82a7-f77098ff164e,afd5ee0e-563c-4238-876a-a1f103037d5b,Adenomas and Adenocarcinomas,False,Kidney,CPTAC-3,Tumor,Illumina HiSeq 4000,NaN,True,TruSeq Stranded Total RNA Library Prep Kit wit...,Illumina,NaN,NaN,rRNA Depletion,First_Stranded,RNA-Seq,342.0,1.0,R1
2,8710ce04-6b7f-4e37-adc0-0df1f2798b30,f0d170a6-dc9d-4334-8a34-3c0ec44fd359,7af5dc64-987e-49b3-a28b-8b17dfb9146c,c8302fca-0062-44b5-b3ee-0f8a13bfa37f,Adenomas and Adenocarcinomas,False,Bronchus and lung,CPTAC-3,Tumor,Illumina HiSeq 4000,NaN,True,TruSeq Stranded Total RNA Library Prep Kit wit...,Illumina,NaN,NaN,rRNA Depletion,First_Stranded,RNA-Seq,305.0,1.0,R1
3,c1898677-7a92-48cf-a09f-71d91c1cf8dc,e7b35aee-bee6-4beb-98ff-70f9b107c7dd,a43ca86d-3d26-412b-89ea-2a73a1bd73ee,b8342ecf-6779-436c-bcfa-55abd631e62e,Ductal and Lobular Neoplasms,False,Pancreas,CPTAC-3,Tumor,Illumina HiSeq 4000,NaN,True,TruSeq Stranded Total RNA Library Prep Kit wit...,Illumina,NaN,NaN,rRNA Depletion,First_Stranded,RNA-Seq,308.0,1.0,R1
4,e1d68cfb-04e7-43cb-b8c5-b523cf917636,c44d7d46-e227-468a-9b5b-30887ba7a6ad,09398641-10ef-4459-963b-9c4652521d18,f2f1e7cc-e192-4e23-adef-69eb17a72ce0,Squamous Cell Neoplasms,False,Other and ill-defined sites,CPTAC-3,Tumor,Illumina HiSeq 4000,NaN,True,TruSeq Stranded Total RNA Library Prep Kit wit...,Illumina,NaN,NaN,rRNA Depletion,First_Stranded,RNA-Seq,309.0,1.0,R1


In [54]:
# preview other dataframe
df2.head()

,case_id,demographic_id,diagnosis_id,batch_id,disease_type,index_date,primary_site,project_id,cause_of_death,days_to_birth,days_to_death,ethnicity,gender,race,vital_status,adrenal_hormone,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_stage,ajcc_clinical_t,ajcc_pathologic_m,ajcc_pathologic_n,ajcc_pathologic_stage,ajcc_pathologic_t,ajcc_staging_system_edition,ann_arbor_b_symptoms,ann_arbor_b_symptoms_described,ann_arbor_clinical_stage,ann_arbor_extranodal_involvement,ann_arbor_pathologic_stage,best_overall_response,burkitt_lymphoma_clinical_variant,child_pugh_classification,classification_of_tumor,cog_liver_stage,cog_neuroblastoma_risk_group,cog_renal_stage,cog_rhabdomyosarcoma_risk_group,created_datetime,days_to_best_overall_response,days_to_diagnosis,days_to_last_follow_up,days_to_last_known_disease_status,days_to_recurrence,diagnosis_is_primary_disease,eln_risk_classification,enneking_msts_grade,enneking_msts_metastasis,enneking_msts_stage,enneking_msts_tumor_site,esophageal_columnar_dysplasia_degree,esophageal_columnar_metaplasia_present,figo_stage,figo_staging_edition_year,first_symptom_prior_to_diagnosis,gastric_esophageal_junction_involvement,gleason_grade_group,gleason_grade_tertiary,gleason_patterns_percent,goblet_cells_columnar_mucosa_present,icd_10_code,igcccg_stage,inpc_grade,inpc_histologic_group,inrg_stage,inss_stage,international_prognostic_index,irs_group,irs_stage,ishak_fibrosis_score,iss_stage,last_known_disease_status,laterality,margin_distance,margins_involved_site,masaoka_stage,medulloblastoma_molecular_classification,metastasis_at_diagnosis,metastasis_at_diagnosis_site,method_of_diagnosis,micropapillary_features,mitosis_karyorrhexis_index,mitotic_count,morphology,ovarian_specimen_status,ovarian_surface_involvement,papillary_renal_cell_type,peritoneal_fluid_cytological_status,pregnant_at_diagnosis,primary_diagnosis,primary_disease,primary_gleason_grade,prior_malignancy,prior_treatment,progression_or_recurrence,project_id.1,residual_disease,satellite_nodule_present,secondary_gleason_grade,site_of_resection_or_biopsy,sites_of_involvement,submitter_id,supratentorial_localization,synchronous_malignancy,tissue_or_organ_of_origin,tumor_confined_to_organ_of_origin,tumor_depth,tumor_focality,tumor_grade,tumor_regression_grade,updated_datetime,weiss_assessment_score,who_cns_grade,who_nte_grade,wilms_tumor_histologic_subtype,year_of_diagnosis
0,f28fc405-1fac-4ee0-b398-e0ddf7b4e639,aa2cc284-5645-40eb-a95a-dbb391711d9b,01282185-756e-42f0-a78c-69afa886f38a,4.0,Squamous Cell Neoplasms,Diagnosis,Bronchus and lung,CPTAC-3,NaN,-27320.0,3.0,not reported,male,white,Dead,NaN,27320.0,NaN,NaN,NaN,NaN,NaN,NaN,Stage IIA,NaN,7th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-03T12:05:16.846188-06:00,NaN,NaN,3.0,3.0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tumor free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8070/3,NaN,NaN,NaN,NaN,NaN,"Squamous cell carcinoma, NOS",NaN,NaN,NaN,NaN,no,CPTAC-3,RX,NaN,NaN,"Middle lobe, lung",NaN,C3L-00445-DIAG,NaN,NaN,"Middle lobe, lung",NaN,NaN,Unifocal,G2,NaN,2021-09-22T14:36:32.037275-05:00,NaN,NaN,NaN,NaN,2017.0
1,dc371c9c-9ea3-4d19-969e-34b9c7b3ada2,fb5a9172-ff61-4d06-adaf-325fe67197f2,b62ec1cd-148b-45df-a6dc-2d908e00e94a,4.0,Squamous Cell Neoplasms,Diagnosis,Bronchus and lung,CPTAC-3,NaN,-21800.0,NaN,not reported,male,other,Alive,NaN,21800.0,NaN,NaN,NaN,NaN,NaN,NaN,Stage IB,NaN,7th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-03T12:05:16.846188-06:00,NaN,NaN,1376.0,1376.0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tumor free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8070/3,NaN,NaN,NaN,NaN,NaN,"Squamous cell carcinoma, NOS",NaN,NaN,NaN,NaN,no,CPTAC-3,R0,NaN,NaN,"Upper lobe, lung",NaN,C3L-01000-DIAG,NaN,NaN,"Upper lobe, lung",NaN,NaN,Unifocal,G3,NaN,2023-02-03T18:28:48.308091-06:00,NaN,NaN,NaN,NaN,2017.0
2,6bfa9

In [55]:
# join both by case_id and keep all rows
df3 = df.merge(df2, how='outer', on='case_id', suffixes=('_bio', '_gdc'))

# get shape of the dataframe
df3.shape

(6706, 138)

In [56]:
# preview joined dataframe
df3.head()

,case_id,aliquot_id,read_group_id,submitted_unaligned_reads_id,disease_type_bio,has_blood_cancer,primary_site_bio,project_id_bio,tissue_type,instrument_model,RIN,includes_spike_ins,library_preparation_kit_name,library_preparation_kit_vendor,library_preparation_kit_version,library_preparation_kit_catalog_number,library_selection,library_strand,library_strategy,size_selection_range,spike_ins_concentration,read_pair_number,demographic_id,diagnosis_id,batch_id,disease_type_gdc,index_date,primary_site_gdc,project_id_gdc,cause_of_death,days_to_birth,days_to_death,ethnicity,gender,race,vital_status,adrenal_hormone,age_at_diagnosis,ajcc_clinical_m,ajcc_clinical_n,ajcc_clinical_stage,ajcc_clinical_t,ajcc_pathologic_m,ajcc_pathologic_n,ajcc_pathologic_stage,ajcc_pathologic_t,ajcc_staging_system_edition,ann_arbor_b_symptoms,ann_arbor_b_symptoms_described,ann_arbor_clinical_stage,ann_arbor_extranodal_involvement,ann_arbor_pathologic_stage,best_overall_response,burkitt_lymphoma_clinical_variant,child_pugh_classification,classification_of_tumor,cog_liver_stage,cog_neuroblastoma_risk_group,cog_renal_stage,cog_rhabdomyosarcoma_risk_group,created_datetime,days_to_best_overall_response,days_to_diagnosis,days_to_last_follow_up,days_to_last_known_disease_status,days_to_recurrence,diagnosis_is_primary_disease,eln_risk_classification,enneking_msts_grade,enneking_msts_metastasis,enneking_msts_stage,enneking_msts_tumor_site,esophageal_columnar_dysplasia_degree,esophageal_columnar_metaplasia_present,figo_stage,figo_staging_edition_year,first_symptom_prior_to_diagnosis,gastric_esophageal_junction_involvement,gleason_grade_group,gleason_grade_tertiary,gleason_patterns_percent,goblet_cells_columnar_mucosa_present,icd_10_code,igcccg_stage,inpc_grade,inpc_histologic_group,inrg_stage,inss_stage,international_prognostic_index,irs_group,irs_stage,ishak_fibrosis_score,iss_stage,last_known_disease_status,laterality,margin_distance,margins_involved_site,masaoka_stage,medulloblastoma_molecular_classification,metastasis_at_diagnosis,metastasis_at_diagnosis_site,method_of_diagnosis,micropapillary_features,mitosis_karyorrhexis_index,mitotic_count,morphology,ovarian_specimen_status,ovarian_surface_involvement,papillary_renal_cell_type,peritoneal_fluid_cytological_status,pregnant_at_diagnosis,primary_diagnosis,primary_disease,primary_gleason_grade,prior_malignancy,prior_treatment,progression_or_recurrence,project_id.1,residual_disease,satellite_nodule_present,secondary_gleason_grade,site_of_resection_or_biopsy,sites_of_involvement,submitter_id,supratentorial_localization,synchronous_malignancy,tissue_or_organ_of_origin,tumor_confined_to_organ_of_origin,tumor_depth,tumor_focality,tumor_grade,tumor_regression_grade,updated_datetime,weiss_assessment_score,who_cns_grade,who_nte_grade,wilms_tumor_histologic_subtype,year_of_diagnosis
0,5705efcc-b48f-435c-8a28-9e0d407ecadd,75ac0619-947a-427b-a53f-71e121a7ec8f,71894d8b-5210-44dc-aadc-a199d3843dd2,8fcdd6a1-a7c7-41b5-8b44-e41f2237b236,Adenomas and Adenocarcinomas,False,"Uterus, NOS",CPTAC-3,Tumor,Illumina HiSeq 4000,NaN,True,TruSeq Stranded Total RNA Library Prep Kit wit...,Illumina,NaN,NaN,rRNA Depletion,First_Stranded,RNA-Seq,308.0,1.0,R1,f1ec7b5b-7f11-41e6-9fb4-a27fb2adc8f1,c7f325cd-98a6-4996-825e-1c23b86e1a22,2.0,Adenomas and Adenocarcinomas,Diagnosis,"Uterus, NOS",CPTAC-3,NaN,-22142.0,NaN,not hispanic or latino,female,white,Alive,NaN,22142.0,M0,NaN,NaN,NaN,Unknown,NX,Stage I,T1a,7th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-17T23:34:39.302537-05:00,NaN,NaN,823.0,823.0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stage I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tumor free,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8380/3,NaN,NaN,NaN,NaN,NaN,"Endometrioid adenocarcinoma, NOS",NaN,NaN,NaN,NaN,no,CPTAC-3,R0,NaN,NaN,Corpus uteri,NaN,C3L-00137-DIAG,NaN,NaN,Corpus uteri,NaN,NaN,Unifocal,G1,NaN,2021-09-22T14:36:32.037275-05:00,NaN,NaN,NaN,NaN,2016.0
1,5705efcc-b48f-435c-8a28-9e0d407ecadd,7

In [57]:
# remove the column read_pair_number and submitted_unaligned_reads_id
df3 = df3.drop(['read_pair_number'], axis=1)
df3 = df3.drop(['submitted_unaligned_reads_id'], axis=1)

# drop duplicated and save as new dataframe
df4 = df3.drop_duplicates()

# print shape of the old and new dataframes
print(df3.shape)
print(df4.shape)

(6706, 136)
(3355, 136)


In [58]:
# check duplicate case_id
df4[df4.duplicated(['case_id'], keep=False)].shape

(2345, 136)

In [59]:
# Iterate over each pair of columns that were repeated in both initial dataframes and combine the values
for col in df4.columns:
    if col.endswith('_bio'):
        gdc_col = col.replace('_bio', '_gdc')
        if gdc_col in df4.columns:
            new_col_name = col.replace('_bio', '')  # Create a new column name without suffixes
            new_column_values = []
            # Iterate down the rows in the DataFrame
            for i in range(len(df4)):
                bio_value = df4[col].iloc[i]
                gdc_value = df4[gdc_col].iloc[i]
                if not pd.isnull(bio_value):
                    new_column_values.append(bio_value)
                else:
                    new_column_values.append(gdc_value)
            df4[new_col_name] = new_column_values  # Add a new column with combined values
            df4.drop([col, gdc_col], axis=1, inplace=True)  # Delete the original column pair

/var/folders/n5/_h8qqt011vq0417yxq9kch240000gq/T/ipykernel_40673/476691515.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4[new_col_name] = new_column_values  # Add a new column with combined values
/var/folders/n5/_h8qqt011vq0417yxq9kch240000gq/T/ipykernel_40673/476691515.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop([col, gdc_col], axis=1, inplace=True)  # Delete the original column pair
/var/folders/n5/_h8qqt011vq0417yxq9kch240000gq/T/ipykernel_40673/476691515.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [60]:
# save the dataframe
df4.to_csv('../../results/metadata.tsv', sep='\t', index=False)

In [61]:
source_dir = "../../data/local_results/"  # Change this to the source directory where your .sf.gz files are located
destination_dir = "../../data/local_results/sf"

# Get a list of all files in the source directory
filenames = os.listdir(source_dir)

for filename in filenames:
    if filename.endswith('.sf.gz'):
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, filename[:-3])

        # Check if the destination file already exists
        if not os.path.exists(destination_path):
            with gzip.open(source_path, 'rb') as f_in:
                with open(destination_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        else:
            print(f"Skipping {filename} as it has already been unzipped.")

Skipping f170a798-ee41-46da-af3f-9392f83bd205.salmon.quant.sf.gz as it has already been unzipped.
Skipping c6ca070f-d1cc-4603-a9f5-7a9f479236ed.salmon.quant.sf.gz as it has already been unzipped.
Skipping dba512d3-58c5-47ed-a3f5-ee3d81e0b0c9.salmon.quant.sf.gz as it has already been unzipped.
Skipping cd2a7dca-e7b3-44ec-82de-68b1ba50fed7.salmon.quant.sf.gz as it has already been unzipped.
Skipping dc8e4126-266e-48c9-8577-e604d108fa4f.salmon.quant.sf.gz as it has already been unzipped.
Skipping b0beb620-f89a-4018-a9d0-6f2b7c8e3c4e.salmon.quant.sf.gz as it has already been unzipped.
Skipping e206721b-2021-41de-b389-878da800c6a0.salmon.quant.sf.gz as it has already been unzipped.
Skipping b3ada7ba-cd7c-476e-bdd4-a774fd5b9d27.salmon.quant.sf.gz as it has already been unzipped.
Skipping 097b8a4f-9036-4223-85a0-7a54aaa8c2e4.salmon.quant.sf.gz as it has already been unzipped.
Skipping a2138f9a-a103-41ee-95c8-1be4489cbf19.salmon.quant.sf.gz as it has already been unzipped.
Skipping bc7641eb-81

In [62]:
# Define the directory where the '.sf' files are located
sf_directory = "../../data/local_results/sf/"

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# List the '.sf' files in the directory
sf_files = [f for f in os.listdir(sf_directory) if f.endswith('.sf')]

# take one .sf file, remove all columns except Name and TPM, and rename TPM to everything before the first period in the filename
# preview = pd.read_csv("../../data/local_results/sf/" + sf_files[0], sep='\t', header=0)
# preview = preview[['Name', 'TPM']]
# preview = preview.rename(columns={'TPM': sf_files[0].split('.')[0]})
# preview.head()

# loop through all .sf files
for file in tqdm(sf_files):
    # read in the file
    df = pd.read_csv("../../data/local_results/sf/" + file, sep='\t', header=0)
    # keep only the Name and TPM columns
    df = df[['Name', 'TPM']]
    # rename the TPM column to the filename without the .sf extension
    df = df.rename(columns={'TPM': file.split('.')[0]})
    # if results_df is empty, then set it to the df dataframe
    if result_df.empty:
        result_df = df
    else:
        # merge the dataframe with the result_df dataframe
        result_df = pd.merge(result_df, df, on='Name', how='outer')

100%|██████████| 133/133 [00:44<00:00,  2.98it/s]


In [63]:
# save the results_df dataframe as a tsv file with the name TPM.tsv
result_df.to_csv('../../results/TPM.tsv', sep='\t', index=False)